In [1]:
# === IMPORTS ===
import os
import sys
import json
import numpy as np
from typing import List, Dict
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict

sys.path.insert(0, '/home/shougan/projects/aip-fredashi/shougan/balance-budget')
BASE_DIR = Path('/home/shougan/projects/aip-fredashi/shougan/balance-budget')

from tuning.config import OUTPUTS_DIR
from tuning.inference.ifeval_inference import run_inference_ifeval
from instruction_following_eval import evaluation_lib, instructions_registry

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-26 11:39:51 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

In [4]:
# === UTILITY FUNCTIONS ===

def pass_at_k(n: int, c: int, k: int) -> float:
    """Calculate pass@k: probability that at least one of k samples is correct."""
    if n - c < k:
        return 1.0
    return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

def compute_pass_at_k_scores(results_per_prompt: List[List[bool]], k_values: List[int]) -> Dict[int, float]:
    """Compute average pass@k across all prompts."""
    scores = {k: [] for k in k_values}
    for results in results_per_prompt:
        n, c = len(results), sum(results)
        for k in k_values:
            if k <= n:
                scores[k].append(pass_at_k(n, c, k))
    return {k: np.mean(v) for k, v in scores.items() if v}

def save_responses(results: List[Dict], model_name: str):
    path = OUTPUTS / model_name
    path.mkdir(parents=True, exist_ok=True)
    with open(path / "responses_multi_sample.jsonl", "w") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def load_responses(model_name: str) -> List[Dict]:
    with open(OUTPUTS / model_name / "responses_multi_sample.jsonl") as f:
        return [json.loads(line) for line in f]

In [5]:
# === CONFIGURATION ===
model_name = "llama3-8B_sft-tuluif-10000"
k_values = [1, 2, 4, 8, 16, 32, 64]
n_samples = 128
temperature = 0.7
run_inference_flag = True
num_examples = 20  # Set to None for full dataset

# Setup paths
OUTPUTS = Path(OUTPUTS_DIR) / "pass@k_responses"
IFEVAL_INPUT_PATH = BASE_DIR / "instruction_following_eval/data/input_data.jsonl"

In [6]:
# === STEP 1: LOAD IFEVAL INPUTS ===
inputs_map = {inp.prompt: inp for inp in evaluation_lib.read_prompt_list(str(IFEVAL_INPUT_PATH))}
print(f"Loaded {len(inputs_map)} IFEval prompts")

Loaded 541 IFEval prompts


In [10]:
print(evaluation_lib.read_prompt_list(str(IFEVAL_INPUT_PATH))[0])
test_prompt = evaluation_lib.read_prompt_list(str(IFEVAL_INPUT_PATH))[0].prompt
print("\n", test_prompt)
print("\n",inputs_map[test_prompt])

InputExample(key=1000, instruction_id_list=['punctuation:no_comma', 'detectable_format:number_highlighted_sections', 'length_constraints:number_words'], prompt='Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.', kwargs=[{}, {'num_highlights': 3}, {'relation': 'at least', 'num_words': 300}])

 Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.

 InputExample(key=1000, instruction_id_list=['punctuation:no_comma', 'detectable_format:number_highlighted_sections', 'length_constraints:number

In [8]:
# === STEP 2: RUN INFERENCE OR LOAD CACHED RESPONSES ===
print(f"Model: {model_name}")

if run_inference_flag:
    print(f"Running inference with n_samples={n_samples}, temperature={temperature}")
    raw_results = run_inference_ifeval(
        model_name=model_name,
        n_samples=n_samples,
        temperature=temperature,
        save_results=False,
        num_examples=num_examples
    )
    # Group responses by prompt for pass@k evaluation
    # raw_results: [{prompt: "", responses: ["", "", ...]}, ...] where each response is separate
    grouped = defaultdict(list)
    for r in raw_results:
        grouped[r["prompt"]].append(r["response"])
    
    model_results = [{"prompt": p, "responses": resps} for p, resps in grouped.items()]
    print(f"Generated {len(model_results)} prompts with {n_samples} samples each")
    save_responses(model_results, model_name)
    print("Responses saved.")
else:
    print("Loading cached responses...")
    model_results = load_responses(model_name)
    print(f"Loaded {len(model_results)} results")

Model: llama3-8B_sft-tuluif-10000
Running inference with n_samples=128, temperature=0.7
Loading model from /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_sft-tuluif-10000
Using GPU memory utilization: 0.8
INFO 01-26 11:40:36 [config.py:841] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate'}. Defaulting to 'generate'.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 01-26 11:40:36 [config.py:1472] Using max model len 131072
INFO 01-26 11:40:39 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 01-26 11:40:39 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 01-26 11:40:45 [__init__.py:244] Automatically detected platform cuda.
INFO 01-26 11:40:47 [core.py:526] Waiting for init message from front-end.
INFO 01-26 11:40:47 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_sft-tuluif-10000', speculative_config=None, tokenizer='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_sft-tuluif-10000', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.57it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.40it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.33it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.78it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.62it/s]



INFO 01-26 11:40:51 [default_loader.py:272] Loading weights took 2.50 seconds
INFO 01-26 11:40:51 [gpu_model_runner.py:1801] Model loading took 15.0006 GiB and 2.759446 seconds
INFO 01-26 11:40:58 [backends.py:508] Using cache directory: /home/shougan/.cache/vllm/torch_compile_cache/e7887f0b3e/rank_0_0/backbone for vLLM's torch.compile
INFO 01-26 11:40:58 [backends.py:519] Dynamo bytecode transform time: 6.89 s
INFO 01-26 11:41:03 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 4.685 s
INFO 01-26 11:41:04 [monitor.py:34] torch.compile takes 6.89 s in total
INFO 01-26 11:41:05 [gpu_worker.py:232] Available KV cache memory: 43.36 GiB
INFO 01-26 11:41:05 [kv_cache_utils.py:716] GPU KV cache size: 355,168 tokens
INFO 01-26 11:41:05 [kv_cache_utils.py:720] Maximum concurrency for 131,072 tokens per request: 2.71x


...

Processed prompts:   0%| | 0/2560 [00:00<?, ?it/s, est. speed

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/python/3.11.5/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/tqdm/_monitor.py", line 84, in run
    instance.refresh(nolock=True)
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/tqdm/notebook.py", line 171, in display
    rtext.value = right
    ^^^^^^^^^^^
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/traitlets/traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/traitlets/traitlets.py", line 706, in set
    obj._notify_trait(self.nam

Generated 20 responses using vllm


[rank0]:[W126 11:44:00.441396819 ProcessGroupNCCL.cpp:1476] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


Generated 20 prompts with 128 samples each
Responses saved.


In [7]:
# === STEP 4: EVALUATE SINGLE RESPONSE USING PRE-BUILT FUNCTIONS ===
def evaluate_single_response(inp: evaluation_lib.InputExample, response: str, strict: bool = True) -> bool:
    """Evaluate a single response using the pre-built IFEval functions."""
    prompt_to_response = {inp.prompt: response}
    
    if strict:
        result = evaluation_lib.test_instruction_following_strict(inp, prompt_to_response)
    else:
        result = evaluation_lib.test_instruction_following_loose(inp, prompt_to_response)
    
    return result.follow_all_instructions

In [8]:
test_response = model_results[0]["responses"][0]
test_eval_input = inputs_map[model_results[0]["prompt"]]
evaluate_single_response(test_eval_input,test_response)

False

In [9]:
# === STEP 5: COMPUTE PASS@K SCORES ===

model_name = "llama3-8B_sft-tuluif-10000"
model_results = load_responses(model_name)
final_results_strict = defaultdict(list)
final_results_loose = defaultdict(list)
# === PIPELINE ===
results_per_prompt = {}
for item in tqdm(model_results, desc="Evaluating responses"):
    prompt = item["prompt"]
    responses = item["responses"]
    eval_input = inputs_map[prompt]
    strict_results = [evaluate_single_response(eval_input, r, strict=True) for r in responses]
    loose_results = [evaluate_single_response(eval_input, r, strict=False) for r in responses]
    results_per_prompt[prompt] = {
        "strict": strict_results,
        "loose": loose_results,
        **{f"strict_k{i}": pass_at_k(len(strict_results), sum(strict_results), i) for i in k_values},
        **{f"loose_k{i}": pass_at_k(len(loose_results), sum(loose_results), i) for i in k_values},
        "strict_k":[pass_at_k(len(strict_results), sum(strict_results), i) for i in k_values],
        "loose_k":[ pass_at_k(len(loose_results), sum(loose_results), i) for i in k_values]
    }
    for i in k_values:
        final_results_strict[f"k{i}"].append(pass_at_k(len(strict_results), sum(strict_results), i))
        final_results_loose[f"k{i}"].append(pass_at_k(len(loose_results), sum(loose_results), i))
    # for r in responses:
    #     if evaluate_single_response(eval_input, r, strict=True):
    #         print(prompt)
    #         print(r)
    #         print("="*50)
    #         break

Evaluating responses: 100%|█| 541/541 [00:53<00:00, 10.18it/s


In [ ]:

print([np.mean(final_results_strict[f"k{i}"]) for i in k_values])


[0.5973313308687616, 0.7070350321274537, 0.7868362086127725, 0.8413829714691288, 0.8791300770301045, 0.9065080487227927, 0.9260628465804066]


In [28]:
my_sum = 0

for prompt, result in results_per_prompt.items():
    my_sum += result['loose_k64']
print(my_sum/541)

0.9574861367837338


In [ ]:
# === FINAL RESULTS ===
print(f"\n{'='*50}")
print(f"RESULTS: {model_name}")
print(f"{'='*50}")
print(f"Strict pass@k: {strict_scores}")
print(f"Loose pass@k:  {loose_scores}")